In [2]:
%load_ext autoreload
%autoreload 2

## Introduction:

The goal of this project is to recover a high resolution image from a low resolution input. In order to accomplish this goal, we will be deploying the super-resolution convolutional neural network (SRCNN) using PyTorch. This network was published in the paper, **Image Super-Resolution Using Deep Convolutional Networks** by *Chao Dong, et al. in 2014*. In addition we implemented FSRCNN, XX and YY for ZZ purposes.

The research paper is attached in the GitHub Repository itself.

Single image super-resolution, which aims at
recovering a high-resolution image from a single low resolution image, is a classical problem in computer
vision, and the researchers named the proposed model Super-Resolution Convolutional Neural Network. (SRCNN)

The SRCNN is a deep convolutional neural network that learns end-toend mapping of resolution to high resolution images. As a result, we can use it to improve the image quality of low resolution images.

In here, we present a fully convolutional neural network for image super-resolution. The network directly learns an end-to-end mapping between low and high-resolution images, with little pre/postprocessing beyond the optimization.

To evaluate the performance of this network, we will be using two image quality metrics:

1. **Peak Signal-To-Noise Ratio(PSNR)**:- It is an expression for the ratio between the maximum possible value (power) of a signal and the power of distorting noise that affects the quality of its representation. The higher the PSNR, the better degraded image has been reconstructed to match the original image.

2. **Structural Similarity Index(SSIM)**:- The Structural SIMilarity (SSIM) index is a method for measuring the similarity between two images. The SSIM index can be viewed as a quality measure of one of the images being compared, provided the other image is regarded as of perfect quality.

Furthermore, we will be using **OpenCV** for *pre* and *post process* our images.

### 1. Importing Packages:

Here we are importing the pakages that we will be using for our project and will list out the version for each one of them.

Mainly we are using these following packages:

1. **sys** -- provides access to some variables used or maintained by the interpreter and to functions that interact strongly with the interpreter.

2. **pytorch** -- is an open-source neural-network library written in Python.

3. **cv2** -- is a library of programming functions mainly aimed at real-time computer vision.

4. **numpy** -- numerical computation library.

5. **matplotlib** -- visualization library.

6. **skimage** -- for image processing.

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import torch

### 2. Preparing Images

For this project we will be using the same images that were used in the original SRCNN paper. All the images are under the source directory.

Now we will produce low resolution versions of these same images and we will place all the low resolution images in Images directory. 

We will accomplish the task for lowering down the resolution by resizing the images, both downwardsa and upwards, using OpenCV and will be using bilinear interpolation.

**Interpolation**:- Interpolation is the insertion of something of a different nature into something else.

**Bilinear Interpolation**:- A weighted average of the attributes (color, alpha, etc.) of the four surrounding pixels is computed and applied to the screen pixel. This process is repeated for each pixel forming the object being textured.

In [ ]:
def prepare_images(path, factor):
    '''
    Function to prepare degraded images by introducing distortions via resizing
    
    Arguments:
    path -- file path where the high-res images are stored.
    factor -- value by which resizing will take place.
    '''
    ## looping through the files in the directory
    for file in os.listdir(path):
        ## open the image
        img = cv2.imread(path + '/' + file)
        
        ## image dimention
        h, w, c = img.shape
        new_height = int(h / factor)
        new_width = int(w / factor)
        
        ## resizing the image - down
        img = cv2.resize(img, (new_width, new_height), interpolation=cv2.INTER_LINEAR)
        
        ## resizing the image - up
        img = cv2.resize(img, (w, h), interpolation=cv2.INTER_LINEAR)
        
        ## save the image
        cv2.imwrite('Images/{}'.format(file), img)
        print('Saving {}'.format(file))

### 3. Testing Low Resolution Images

To ensure that our image quality metrics are being calculated correctly and that the images were effectively degraded. we will now calculate the PSNR, MSE and SSIM between our reference images and the drgraded images that we just prepared.

In [ ]:
## testing the generated images using image quality metrics
for file in os.listdir('Images'):
    ## high-res images
    target = cv2.imread('source/{}'.format(file))
    
    ## low-res images
    ref = cv2.imread('Images/{}'.format(file))
    
    ## calculate the scores
    scores = compare_images(target, ref)
    
    ## display the results
    print('{}\nPSNR: {}\nMSE: {}\nSSIM: {}\n'.format(file, scores[0], scores[1], scores[2]))

### 4. Building the Models

Now that we have our low resolution images and all three image quailty metrics are working properly. Now we will start building the SRCNN model. 

In [ ]:
import models

srcnn = models.SRCNN()

Image of SRCNN Architecture and Explanation of Components